<a href="https://colab.research.google.com/github/thalitadru/ml-class-epf/blob/main/LabSession4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial on ensamble learning with scikit-learn

*Credits:* This tutorial is based on notebooks written by [A. Géron](https://colab.research.google.com/github/ageron/handson-ml2/) for his "Hands-on ML" book. Code realeased under [Apache 2.0 license](https://github.com/ageron/handson-ml2/blob/master/LICENSE).

In [ ]:
import matplotlib.pyplot as plt
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
from scipy import stats


## Some useful plotting functions

In [ ]:
from matplotlib.colors import ListedColormap

def plot_data(X, y, ax=None, axes=[-1.5, 2.45, -1, 1.5], alpha=0.5,):
    if ax is None:
        ax = plt.gcf().gca()
    ax.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", alpha=alpha)
    ax.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", alpha=alpha)
    ax.axis(axes)
    ax.set_xlabel(r"$x_1$", fontsize=18)
    ax.set_ylabel(r"$x_2$", fontsize=18, rotation=0)

def plot_decision_boundary(clf, X, y, ax=None, axes=[-1.5, 2.45, -1, 1.5], alpha=0.5, contour=True):
    if ax is None:
        ax = plt.gcf().gca()
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    ax.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if contour:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        ax.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    plot_data(X, y, ax=ax, axes=axes, alpha=alpha)


# A few experiments with ensembles on toy data

Let's use the moons dataset:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
plot_data(X_train, y_train)

## Voting ensemble classifier

First we import and instantiate the induvisual classifiers constituting our ensemble:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42)


Then, we can use [`VotingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html#sklearn.ensemble.VotingClassifier) to combine the individual estimators:

In [ ]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')

Calling fit on the voting classifier will train the individual models:

In [ ]:
voting_clf.fit(X_train, y_train)

We can access the individual estimators trained for the ensamble through the attribute `estimators_`:

In [ ]:
voting_clf.estimators_

In [ ]:
from sklearn.metrics import accuracy_score
fig, axs = plt.subplots(1, 4, figsize=(20,4))
for i, clf in enumerate((log_clf, rnd_clf, svm_clf, voting_clf)):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
    ax = axs[i]
    plot_decision_boundary(clf, X_train, y_train, ax=ax)
    ax.set_title(clf.__class__.__name__)

### Diversity within the ensemble
Notice how the models make different mistakes:

In [ ]:
individual_miss = set()
for model in voting_clf.estimators_:
    y_pred = model.predict(X_train)
    miss_ids = np.arange(X_train.shape[0])[y_pred != y_train]
    individual_miss = individual_miss.union(set(miss_ids))
    print(f"Mistakes on training set for model {model.__class__.__name__}: \nsamples {miss_ids}")

The ensemble model allows to combine stengths and compensate weakeness of the composing models, finally making less mistakes

In [ ]:
y_pred = voting_clf.predict(X_train)
miss_ids = np.arange(X_train.shape[0])[y_pred != y_train]
ensemble_miss = set(miss_ids)

print(f'union of individual mistakes: {len(individual_miss)} samples\n {sorted(individual_miss)}')
print(f'ensemble mistakes: {len(ensemble_miss)} samples\n {sorted(ensemble_miss)}' )

Here you can see the predictions of each individual model for all the training samples over which one of them made mistakes

In [ ]:
# pd.set_option('display.max_rows', 20)
ids = np.array(sorted(individual_miss))
df = pd.DataFrame(index=ids)
for model in voting_clf.estimators_:
    y_pred = model.predict(X_train[ids])
    df[model.__class__.__name__] = y_pred
df[voting_clf.__class__.__name__] = voting_clf.predict(X_train[ids])


The prediction output by the voting ensamble is computed by simple majority voting, as verified by the assert statement bellow:

In [ ]:
df['Majority'] = df.aggregate(func=lambda x: stats.mode(x)[0][0], axis=1)
assert((df['VotingClassifier'] == df['Majority']).all())
df

### Soft-Voting 
The same class `VotingClassifier` can be used to combine estimators through a weighted voting method. 
In this case, the voting is done by averaging the predicted class distributions of each individual classifier.

Do achieve it, just pass the argument `voting='soft'` to the constructor: 



In [ ]:
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", probability=True, random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')
voting_clf.fit(X_train, y_train)

In [ ]:
voting_clf.score(X_test, y_test)

Here are some plots to visualize the predicted probabilities for each indivisual classifier. Note that if we average them manually, we obtain the same response as the `VotingClassifier` we just trained.

In [ ]:
individual_miss = set()
for model in voting_clf.estimators_:
    y_pred = model.predict(X_train)
    miss_ids = np.arange(X_train.shape[0])[y_pred != y_train]
    individual_miss = individual_miss.union(set(miss_ids))


fig, axs = plt.subplots(1, 5, figsize=(20,4))
def plot_class_probas(y_pred_probas, name, ax):
    mp = ax.imshow(y_pred_probas, vmin=0, vmax=1, cmap='coolwarm')
    ax.set_title(name)
    ax.set_xticks([0,1])
    ax.set_xlabel("class id")
    ax.set_ylabel("samples")
    ax.axis("tight")
    return mp

ids = np.array(sorted(individual_miss))[:10]
y_pred_agg = np.zeros((len(ids),2), dtype=float)
for i, model in enumerate(voting_clf.estimators_+[voting_clf]):
    y_pred = model.predict_proba(X_train[ids])
    if i<3:
        y_pred_agg += y_pred/3.
    ax = axs[i]
    plot_class_probas(y_pred, model.__class__.__name__, ax )

mp = plot_class_probas(y_pred_agg, 'Average\nof predicted probas', axs[-1])
fig.colorbar(mp)


## Stacking ensemble
Stacking ensembles or model stacking is the practice of learning multiple estimators and combine them ina smarter way. Instead of simply doing voting or averaging the predictions, we train a model to combine this estimators on separate hold-out set. This model is sometimes called a *blender* model or a *meta-learner*.

Here is an example using a linear model as a blender. First we need to separate a hold-out split from our training data:

In [ ]:
X_train_train, X_train_blend, y_train_train, y_train_blend  = train_test_split(X_train, y_train, test_size=0.3)

Now we train the estimators composing our ensemble on the training subset **without the held-out split**.

In [ ]:
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42)

for clf in (log_clf, rnd_clf, svm_clf):
    clf.fit(X_train_train, y_train_train)

Now we are going to use the hold-out split predictions of each individual estimator on the held-out set as training data for oure blending model.

First, let us gather this data by making predictions:

In [ ]:
preds = {}
for clf in (log_clf, rnd_clf, svm_clf):
    preds[clf.__class__.__name__] = clf.predict(X_train_blend)
df = pd.DataFrame(preds)
df

This data can now be used to train our linear blender (we will use a logistic regression since we are doing classification). 

In [ ]:
from sklearn.linear_model import LogisticRegression
blender = LogisticRegression(random_state=42)
blender.fit(df, y_train_blend)

After fitting, the model has learned intercept and coeficients to combine the predictions of composing estimators:

In [ ]:
blender.coef_, blender.intercept_

Finally, we can combine models and blender to compute predictions and the accuracy score on the test set:

In [ ]:
preds = {}
for clf in (log_clf, rnd_clf, svm_clf):
    preds[clf.__class__.__name__] = clf.predict(X_test)
df = pd.DataFrame(preds)
blender.score(df, y_test)

This performance is not realy better than that of the voting ensemble. There are some factors that could be at cause:
- stacking classifiers works better when combining the predictied probabilities instead of class labels. 
- the size of the held-out dataset may not be big enough to generate sufficient samples for training the blender. 
- the individual estimators were trained on a smaller training set. 
- the linear model is not complex enough to learn a good blending function

We could train the estimators on the full training set, and do the hold-out split directly on the prediction data which is used to train the final model. 
More over, we can use cross-validation to generate multiple hold-out splits and train the blender with better confidence.
We can also try to use a more complex model as a blender.

All of this can be achieved using the classes [`Stackingclassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html) or `StackingRegressor`. They will train the estimators on the full training data and train the final blender using cross validation.

The default blender for classification is LogisticRegression. Notice how we get a similar score:

In [ ]:
from sklearn.ensemble import StackingClassifier

stack = StackingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    cv=5)
stack.fit(X_train, y_train)
stack.score(X_test, y_test)

In [ ]:
stack.final_estimator_.coef_, stack.final_estimator_.intercept_

Now let's use random forests as a blender:

In [ ]:
stack = StackingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    final_estimator=RandomForestClassifier(n_estimators=100, random_state=42), cv=5)
stack.fit(X_train, y_train)
stack.score(X_test, y_test)

## Bagging

Bagging classifiers can be created with `BaggingClassifier`. We need to pass another base estimator that will be fit by the constructor. Here is an example using decision trees:

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, random_state=42)


When we call `fit` on the `BaggingClassifier` object, it will fit `n_estimator=500` trees on boostrapped subsets of the training data :

In [ ]:
bag_clf.fit(X_train, y_train)

Now we can use the bagging ensemble to make predictions on the test set

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
print("Test accuracy for the bagged ensemble of decision trees", accuracy_score(y_test, y_pred))

We can access the individual estimators trained for the ensamble through the attribute `estimators_`. This box plot sumarizes the test performance of the 500 individual trees. As a reference, the green line marks the test performance of the bagged ensemble of these same trees.

In [ ]:
test_acc = []
for estim in bag_clf.estimators_:
    test_acc.append(estim.score(X_test, y_test))
plt.boxplot(test_acc);
plt.xticks([1],['accuracy of individual trees'])
plt.hlines(bag_clf.score(X_test, y_test), 0.5, 1.5, color='g', label='Accuracy of\nbagged ensemble')
plt.legend()

Here is an example comparison of a single decision tree against the bagged ensemble we just trained:

In [ ]:
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print("Test accuracy for a single decision tree", accuracy_score(y_test, y_pred_tree))

y_pred = bag_clf.predict(X_test)
print("Test accuracy for the bagged ensemble of decision trees", accuracy_score(y_test, y_pred))

In [ ]:
fix, axes = plt.subplots(ncols=2, figsize=(10,4), sharey=True)
plt.sca(axes[0])
plot_decision_boundary(tree_clf, X, y)
plt.title("Decision Tree", fontsize=14)
plt.sca(axes[1])
plot_decision_boundary(bag_clf, X, y)
plt.title("Decision Trees with Bagging", fontsize=14)
plt.ylabel("")

### Out-of-bag evaluation

Notice that in bagging, since each classifier is trained on a subsample fo the training set, there are always some unused training samples, that we call *out-of-bag*.
These samples can be used to evaluate the generalization of that individual classifier: this is called the *out-of-bag score* (or oob score) .

Averaging out-of-bag scores for all estimators can be a shortcut to estimating test performance without having to hold out and evaluate on separate validation set.

We can ask Scikit-learn to compute this average oob score as follows:


In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, oob_score=True, random_state=40)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_.round(3)

Notice it is not too far from the test performance

In [ ]:
bag_clf.score(X_test, y_test).round(3)